# k-Nearest Neighbors - Solutions

<hr style="clear:both">

This notebook was developed for the CS-233 Introduction to Machine Learning course at EPFL, adapted for the CIVIL-226 Introduction to Machine Learning for Engineers course, and re-adapted for the ME-390.
We thank contributers in CS-233 ([CVLab](https://www.epfl.ch/labs/cvlab)) and CIVIL-226 ([VITA](https://www.epfl.ch/labs/vita/)).
    
**Author(s):** Jan Bednarík, minor changes by Tom Winandy
<hr style="clear:both">

In this exercise, you will implement a k-Nearest Neighbors (k-NN) classifier and see how to use a validation set to find optimal hyper-parameters.

In [ ]:
# Function to align all tables to the left (useful for later on)

In [ ]:
%%html
<style>
table {float:left}
</style>

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import Any, Callable

# Helper file with functions for pre-processing and visualization
import helpers

## 0. Intro

*Background: You and your friends went on an expedition to the Antarctic, where you befriended a group of penguins. After taking a few measurements, you went back home. You're now wondering if it is possible to classify the penguins you befriended by building a classifier.*

To build such a classifier, you will need some labeled data. Luckily for you, the Palmer Penguins dataset contains exactly the data you need.

You decide to implement a kNN classifier. As a reminder, it works in the following way: For an unknown data point, find the k nearest neighbors from a labeled dataset, and return the majority vote (most frequent class) of these neighbors.

![penguins](images/penguins.jpg)

## 1. Data loading & pre-processing

In this exercise, you will use the [Palmer Penguins dataset](https://github.com/allisonhorst/palmerpenguins) and your goal is to classify 3 species of Penguins: Adélie, Chinstrap and Gentoo.

Let's load the dataset using the [pandas](https://pandas.pydata.org/) library. Don't worry, there's no need to know how to use this library for this exercise.

In [ ]:
# In Pandas, a data table is called a DataFrame (abbreviated to df)
labeled_df = pd.read_csv('data/labeled_penguins.csv')

print(f"There are {labeled_df.shape[0]} rows and {labeled_df.shape[1]} columns.")
# Show the first 5 rows of the data
labeled_df.head(5)

Now, we'll use the function `preprocess_data()` from `helpers.py` to shuffle the data, convert the labels to integers, split it into a training, validation and test set, and then return the result as NumPy arrays.
Feel free to read the code in `helpers.py` to better understand what this function does (completely optional).

In [ ]:
# We'll use 60% of our data as training data, 20% for our validation data and the remaining 20% as test data
# Here, we use a random seed to ensure that the data shuffling and splitting can be reproduced
X_train, y_train, X_val, y_val, X_test, y_test, feature_names, label_map = helpers.preprocess_data(df=labeled_df, label="species", train_size=0.6, val_size=0.2, seed=42)


Run the following cells to better understand how the preprocessed data looks like.

In [ ]:
print(f"Features: {feature_names}")
print(f"Map from label to label name: {label_map}")

In [ ]:
# Visualisation of X_train and y_train (separation of the features and the labels)
print('Training set features (first 10 samples):')
print(f'X_train: \n {X_train[:10]}')

print('\nTraining set labels (first 10 samples):')
print(f'y_train: \n {y_train[:10]}')

In [ ]:
# Show shapes
print('Training set shape:')
print(f'X: {X_train.shape}, y: {y_train.shape}')

print('\nValidation set shape:')
print(f'X: {X_val.shape}, y: {y_val.shape}')

print('\nTest set shape:')
print(f'X: {X_test.shape}, y: {y_test.shape}')

Let's also load our unlabeled measurements of penguins. This is the data we want to classify.

In [ ]:
unlabeled_df = pd.read_csv('data/unlabeled_penguins.csv')

print(f"There are {unlabeled_df.shape[0]} rows and {unlabeled_df.shape[1]} columns.")
# Show the first 5 rows of the data
unlabeled_df.head(5)

In [ ]:
# Convert this unlabeled data to a matrix
X_unlabeled = unlabeled_df.to_numpy()

print('Unlabeled dataset shape:')
print(f'X: {X_unlabeled.shape}')

To summarize, we have an unlabeled dataset of penguins (`X_unlabeled`) that we want to classify. 

We made the assumption that the labeled Palmer Penguins dataset comes from a similar distribution, so we decided to use it for our  classifier. We then split this dataset into a training set (`X_train`, `y_train`), a validation set (`X_val`, `y_val`) and a test set (`X_test`, `y_test`).

## 2. Data visualization & scaling

Let's take a look at our data using the `plot_labeled()` and `plot_unlabeled()` functions of `helpers.py`.

In [ ]:
# Plot training data with labels
helpers.plot_labeled(X_train, y_train, label_map, feature_names, title="Training data")
# Plot validation data with labels
helpers.plot_labeled(X_val, y_val, label_map, feature_names, title="Validation data")
# Plot test data with labels
helpers.plot_labeled(X_test, y_test, label_map, feature_names, title="Test data")
# Plot unlabeled data
helpers.plot_unlabeled(X_unlabeled, feature_names, title="Unlabeled data")

**Question:** What do you notice? How will this affect the kNN classifier?

**Answer:** The body mass and bill length features have drastically different scales. As kNN relies on distance for classification, if these scales are kept, then the bill length feature will have almost no impact on the classifier, which can considerably impact the classifier's accuracy.

As features can represent different physical units and come in vastly different scales, it is common practice to normalize the  data before using kNN.

Here, we'll use Z-score standardization.  That is, for each feature, compute: $x_{norm} = \frac {x - \mu_x} {\sigma_x}$

Let's implement the function `normalize()`:

**Note:** Remember that you should not use any of the knowledge you get from the test data when implementing a model. This includes the normalization step, where you should use the mean and standard deviation of the **training set** to normalize all the other datasets.

In [ ]:
# Compute the mean and standard deviation for each feature of the training set

### START CODE HERE ### (≈ 2 lines of code)
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
### END CODE HERE ###


# Implement the normalize function
def normalize(X: np.ndarray, mean: np.ndarray, std: np.ndarray):
    """ Normalization of array using Z-score standardization
     Args:
        X: Dataset of shape (N, D)
        mean: Mean of shape (D, )
        std: Standard deviation of shape(D, )
    """
    ### START CODE HERE ### (≈ 1 line of code)
    X_normalized = (X - mean) / std
    ### END CODE HERE ###
    return X_normalized

# Normalize features of the training, val and test set using the mean and std of the training set features
X_train = normalize(X_train, mean, std)
X_val = normalize(X_val, mean, std)
X_test = normalize(X_test, mean, std)
X_unlabeled = normalize(X_unlabeled, mean, std)

# Let's rename the features to indicate that they've been normalized
feature_names = ['Bill length (normalized)', 'Body mass (normalized)']

In [ ]:
# Verify that normalization is implemented correctly 
# We check that the mean and std of each feature of the training set is very close to 0 and 1 respectively
if (np.allclose(X_train.mean(axis=0), np.zeros_like(X_train)) and (np.allclose(X_train.std(axis=0), np.ones_like(X_train)))):
    print("All good!")
else:
    print("normalize() doesn't work correctly :(")

Now that the data is normalized, let's plot it again, using the same functions from `helpers` as before.

In [ ]:
# Plot normalized training data with labeled, and normalized test data without labels

### START CODE HERE ### (≈ 4 lines of code)
helpers.plot_labeled(X_train, y_train, label_map, feature_names, title="Training data")
helpers.plot_labeled(X_val, y_val, label_map, feature_names, title="Validation data")
helpers.plot_labeled(X_test, y_test, label_map, feature_names, title="Test data")
helpers.plot_unlabeled(X_unlabeled, feature_names, title="Unlabeled data")
### END CODE HERE ###

Wow, this is much better! Now that the data is normalized, we can implement our classifier.

## 3. Distance metrics

In this part, we'll work on the distance metrics used by the classifier. We have seen two distance metrics:

**The L1 (Manhattan) distance:**

$$
d(\mathbf{p}, \mathbf{q})=\sum_{i=1}^{n} | p_{i}-q_{i} |
$$

It is also sometimes known as city block or taxicab distance, in allusion to the fact that the shortest path by car in a city with a grid layout (such as Manhattan) would result in this distance.

**The L2 (Euclidean) distance:**

$$
d(\mathbf{p}, \mathbf{q})=\sqrt{\sum_{i=1}^{n}\left(p_{i}-q_{i}\right)^{2}}
$$

This is the distance with which you are probably most familar with, it can be seen as the straight line distance between two points.

![distances](images/manhattan_euclidean.png)


Let's implement functions that compute the distance between a single vector to all vectors of the training data. Until the end of this exercise, we define "sample" as a single entry of the dataset. The entry is a vector of D attributes, and the code should stay flexible for any number of attributes.

**Hint:** NumPy's [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html) is your friend.

In [ ]:
def manhattan_dist(sample: np.ndarray, X: np.ndarray):
    """Computes the Manhattan distance between a sample and the training features

    Args:
        sample: Sample of shape (D, )
        X: Dataset of shape (N, D)

    Returns:
        dist: Distances of shape (N,)

    """
    ### START CODE HERE ### (≈ 1 line of code)
    distances = np.abs(X - sample).sum(axis = 1)
    ### END CODE HERE ### 
    return distances
   
def euclidean_dist(sample: np.ndarray, X: np.ndarray):
    """Computes the Euclidean distance between a sample and the training features

    Args:
        sample: Sample of shape (D, )
        X: Dataset of shape (N, D)

    Returns:
        dist: Distances of shape (N,)

    """
    ### START CODE HERE ### (≈ 1 line of code)
    distances = np.sqrt(((X - sample) ** 2).sum(axis = 1))
    ### END CODE HERE ### 
    return distances
   

In [ ]:
# Verify implementation
print(f'Manhattan: {np.round(manhattan_dist(X_val[0], X_train[:3]), decimals=1)}')
print(f'Euclidean: {np.round(euclidean_dist(X_val[0], X_train[:3]), decimals=1)}')

**Expected output:**  

|   |                                                  |
|---|--------------------------------------------------|
| **Manhattan** | [4.2 2.7 2.2] |
| **Euclidean** | [3.1 2.  2. ] |

## 4. Finding neighbors

It's time to find the nearest neighbors.
Implement `find_nearest_neighbors()` according to its documentation.

**Hint:** [`np.argsort()`](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html) is a very useful function when working with indices.

In [ ]:
def find_nearest_neighbors(
    sample: np.ndarray, 
    X: np.ndarray, 
    distance_fn: Callable = euclidean_dist, 
    k: int = 1):
    """Finds the indices of the k-Nearest Neighbors to a sample
    Args:
        sample: Sample of shape (D, )
        X: Dataset of shape (N, D)
        distance_fn: Distance function
        k: Number of nearest neighbors

    Returns:
        indices: Neighbor indices of shape (k, )
    """
    ### START CODE HERE ### (≈ 2 lines of code)
    distances = distance_fn(sample, X)
    neighbor_indices = np.argsort(distances)[:k]
    ### END CODE HERE ###
    
    return neighbor_indices

In [ ]:
# Try changing the sample position, distance function and value of k
# Does your find_nearest_neighbors() implementation seem correct?
sample = np.array([-0.5, -1.5])
distance_fn = euclidean_dist
k = 3

# Plot the nearest neighbors using helpers.py
neighbor_indices = find_nearest_neighbors(sample, X_train, distance_fn=distance_fn, k=k)
helpers.plot_nearest_neighbors(sample, X_train, y_train, neighbor_indices, label_map, feature_names)

The opaque points correspond to the nearest neighbors to the unknown point (star).

Now that you have a function that finds the nearest neighbor, let's implement the function `predict_single()` which returns the majority class of the k-nearest neighbors of a given sample. Do not worry about tie-breakers (i.e. if multiple classes have as many points in proximity to a sample).

**Hints:**
- Feel free to reuse functions you implemented previously
- Labels are numbered from 0 to n-1 (with n the number of classes) 
- [`np.bincount()`](https://numpy.org/doc/stable/reference/generated/numpy.bincount.html) and [`np.argmax()`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) can be very useful.

In [ ]:
def predict_single(
    sample: np.ndarray, 
    X: np.ndarray, 
    y: np.ndarray, 
    distance_fn: Callable, 
    k: int = 1):
    """ Finds the k-Nearest Neighbors to a sample and returns the majority class
    Args:
        sample: Sample of shape (D, )
        X: Dataset of shape (N, D)
        y: Labels of shape (N, )
        distance_fn: Distance function
        k: number of nearest neighbors

    Returns:
        label: Predicted label, the majority class of the k-Nearest Neighbors

    """
    
    ### START CODE HERE ### (≈ 4 lines of code)
    # From your single sample, get the k closest neighbors among X, using your defined distance function:
    neighbor_indices = find_nearest_neighbors(sample, X, distance_fn, k)
    # You have the indice list among the dimension data, get their label from the label data:
    neighbor_labels = y[neighbor_indices]
    # Now count how many neighbors you have for each label:
    neighbor_label_count = np.bincount(neighbor_labels)
    # Get the indice of the most counted label. This is your KNN solution:
    predicted_label = np.argmax(neighbor_label_count)
    ### END CODE HERE ### 
   
    return predicted_label

Now, we can reuse `predict_single()` to predict several samples at once. 

**Hint:** There are several ways to do this, such as using a simple for-loop or with the function [`np.apply_along_axis()`](https://numpy.org/doc/stable/reference/generated/numpy.apply_along_axis.html)

In [ ]:
# With for-loop
def predict(
    samples: np.ndarray, 
    X: np.ndarray = X_train, 
    y: np.ndarray = y_train, 
    distance_fn: Callable = euclidean_dist, 
    k: int = 1):
    """ Finds the k-Nearest Neighbors to a matrix of samples and returns 
        the majority class for each of these samples as an array
    Args:
        samples: Samples of shape (M, D)
        X: Dataset of shape (N, D)
        y: Labels of shape (N, )
        distance_fn: Distance function
        k: number of nearest neighbors

    Returns:
        labels: Predicted labels of shape (M, )

    """
    ### START CODE HERE ### (≈ 1 to 5 lines of code depending on the implementation)
    predicted_labels = []
    for i in range(samples.shape[0]):
        predicted_label = predict_single(samples[i, :], X, y, distance_fn, k)
        predicted_labels.append(predicted_label)
    # Convert to np.ndarray
    predicted_labels = np.array(predicted_labels)
     ### END CODE HERE ###
    
    return predicted_labels

# With apply-along-axis
def predict_1(
    samples: np.ndarray, 
    X: np.ndarray = X_train, 
    y: np.ndarray = y_train, 
    distance_fn: Callable = euclidean_dist, 
    k: int = 1):
    """ Finds the k-Nearest Neighbors to a matrix of samples and returns 
        the majority class for each of these samples as an array
    Args:
        samples: Samples of shape (M, D)
        X: Dataset of shape (N, D)
        y: Labels of shape (N, )
        distance_fn: Distance function
        k: number of nearest neighbors

    Returns:
        labels: Predicted labels of shape (M, )

    """
    ### START CODE HERE ### (≈ 1 to 5 lines of code depending on the implementation)
    predicted_labels = np.apply_along_axis(predict_single, axis=1, arr=samples,
                                           X=X, y=y, distance_fn=distance_fn, k=k)
    ### END CODE HERE ### 
   
    return predicted_labels

In [ ]:
# Verify implementation
print(f'Predicted labels: {predict(X_val[:6], X_train, y_train, euclidean_dist, 5)}')

**Expected output:**  

|   |                                                  |
|---|--------------------------------------------------|
| **Predicted labels** | [1 1 2 1 2 0] |

Let's visualize what predictions look like on our validation set. 
In the following plot, stars represent the validation data (and the color represents their predicted class).

In [ ]:
# Try changing the distance function and value of k
distance_fn = euclidean_dist
k = 3

# Plot the predictions using helpers.py
predicted_val_labels = predict(X_val, X_train, y_train, distance_fn, k=k)
helpers.plot_predictions(samples=X_val, predicted_labels=predicted_val_labels, X=X_train, y=y_train, 
                         label_map=label_map, feature_names=feature_names, 
                         title="Predicted class for validation set")

## 5. Selecting hyper-parameters

Now is the time to select hyper-parameters. For kNN, the two parameters we can select is the distance function and the number of neighbors k.

First, we need a metric to measure how well our model is doing. 

Here, we'll use the accuracy, defined as:
$$ \text{accuracy} = \frac{\text{# correct predictions}}{\text{# all predictions}}$$

Let's implement this function:

In [ ]:
def accuracy(labels_gt: np.ndarray, labels_pred: np.ndarray):
    """ Computes accuracy.
    
    Args:
        labels_gt: labels (ground-truth) of shape (M, ).
        labels_pred: Predicted labels of shape (M, ).
        
    Returns:
        float: Accuracy, in range [0, 1].
    """
    ### START CODE HERE ### (≈ 1 line of code)
    acc = np.sum(labels_gt == labels_pred) / labels_gt.shape[0]
    # Alternative (nicer) formula:
    # acc = np.mean(labels_gt == labels_pred)
    ### END CODE HERE ###
    return acc

Now, let's compute the accuracy for both distance functions at all values of k. We can only afford to do that because our dataset is relatively small, so this exhaustive search isn't too computationally expensive.

In [ ]:
k_values = []
euclidean_accs = []
manhattan_accs = []
for i in range(1, len(X_train) + 1):
    ### START CODE HERE ### (≈ 2 lines of code)
    manhattan_acc = accuracy(y_val, predict(X_val, distance_fn=manhattan_dist, k=i))
    euclidean_acc = accuracy(y_val, predict(X_val, distance_fn=euclidean_dist, k=i))
    ### END CODE HERE ###
    k_values.append(i)
    manhattan_accs.append(manhattan_acc)
    euclidean_accs.append(euclidean_acc)


In [ ]:
# Plot accuracies
plt.figure(figsize=(8,4), dpi=100)
plt.plot(k_values, manhattan_accs, label="Manhattan", alpha=0.8)
plt.plot(k_values, euclidean_accs, label="Euclidean", alpha=0.8)
plt.xlabel("Value of k")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Performance on the validation set for different values of $k$ and distance functions")
plt.show()

**Question:** What do you notice at the highest value of k? How can this be explained?

**Answer:** The accuracy drops dramatically at the highest value of k. This is because, when k is equal to the number of examples in the training data, the kNN classifier returns the majority class of the entire dataset.

The classifier seems to perform well for both distance metrics and for values of k ranging from 1 to 50, but it is hard to pinpoint which value of k is optimal, and which distance metric is better. This is why, for small datasets, it is often preferred to use [K-Fold Cross Validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation) instead of just a single validation set, as it provides more accurate results and is less sensitive to noise. You will see how to implement cross-validation later in this course. For now, just select a value of k and a distance metric that provides good result.

In [ ]:
# Choose a good value of k and a good distance metric
k = 25
distance_fn = euclidean_dist

Now that we selected  good hyperparameters, it is time to check how well our model performs on the test set.

In [ ]:
# Compute the test accuracy
### START CODE HERE ### (≈ 1 line of code)
test_accuracy = accuracy(y_test, predict(X_test, distance_fn=distance_fn, k=k))
### END CODE HERE ###
print(f"Test accuracy: {test_accuracy * 100: .2f}%")

## 6. Final results

It's finally time to use our newly built classifier to assign classes to the unlabeled data.

In [ ]:
# Predict classes of unlabeled data
### START CODE HERE ### (≈ 1 line of code)
predicted_labels = predict(X_unlabeled, distance_fn=distance_fn, k=k)
### END CODE HERE ###


In [ ]:
# Plot predictions
helpers.plot_predictions(samples=X_unlabeled, predicted_labels=predicted_labels, X=X_train, y=y_train, 
                         label_map=label_map, feature_names=feature_names, 
                         title="Predicted class for unlabeled data")

**Question:** What are your thoughts on this class assignment? Do these results look reasonable?

**Answer:** Assuming this unlabeled data comes from the same distribution as the training, validation and test data, these predictions seem mostly correct.

Good job on completing this exercise!